In [1]:
from google.genai import types
from google.adk.events import Event
from collections.abc import AsyncGenerator
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.agents import Agent, BaseAgent, ParallelAgent
from google.adk.agents.invocation_context import InvocationContext

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [3]:
idea_agent = Agent(
    name="IdeaAgent",
    model=model,
    instruction="""
    You are a creative idea generator for short videos for social media.
    Create unique and innovative idea based on user prompts.
    Return only one idea, concise and engaging, in a single sentence.
    """,
    output_key="idea"
)

In [4]:
validation_agent = Agent(
    name="ValidationAgent",
    model=model,
    instruction="""
    You are a video idea validator for short videos on social media targeted to kids.
    Based on the provided idea:
    - If the idea is appropriate for small children, you MUST respond with 'APPROPRIATE'.
    - Otherwise, you MUST respond with 'INAPPROPRIATE'.
    Idea: {idea}
    """,
    output_key="validation_result"
)

In [5]:
script_agent = Agent(
    name="ScriptAgent",
    model=model,
    instruction="""
    You are a video script generator for short videos for social media.
    Based on the provided idea, create an engaging and concise script suitable for a short video format.
    """,
)

In [6]:
actors_agent = Agent(
    name="ActorsAgent",
    model=model,
    instruction="""
    You are an actor head hunter for short videos for social media.
    Based on the provided idea, suggest a list of suitable actors who would fit well in the video.
    """,
)

In [7]:
class CustomAgent(BaseAgent):
    idea_agent: Agent
    validation_agent: Agent
    script_agent: Agent
    actors_agent: Agent
    parallel_agent: ParallelAgent

    def __init__(self, name: str, idea_agent: Agent, validation_agent: Agent, script_agent: Agent, actors_agent: Agent):
        parallel_agent = ParallelAgent(name="ScriptActorsAgent", sub_agents=[script_agent, actors_agent])
        sub_agents_list = [idea_agent, validation_agent, parallel_agent]
        super().__init__(
            name=name,
            idea_agent=idea_agent,
            validation_agent=validation_agent,
            script_agent=script_agent,
            actors_agent=actors_agent,
            parallel_agent=parallel_agent,
            sub_agents=sub_agents_list,
        )

    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        # generate idea
        async for event in self.idea_agent.run_async(ctx):
            yield event
        
        # check if idea was generated
        if "idea" not in ctx.session.state or not ctx.session.state["idea"]:
            return  # stop pipeline
        
        # validate idea
        async for event in self.validation_agent.run_async(ctx):
            yield event
            
        # conditional logic based on validation
        validation_result = ctx.session.state.get("validation_result")
        if validation_result == "INAPPROPRIATE":
            return  # stop pipeline

        # parallell processing (actors and script generation)
        async for event in self.parallel_agent.run_async(ctx):
            yield event


In [8]:
orchiestrator_agent = CustomAgent(
    name="CustomPipeline",
    idea_agent=idea_agent,
    validation_agent=validation_agent,
    script_agent=script_agent,
    actors_agent=actors_agent,
)

In [9]:
runner = InMemoryRunner(agent=orchiestrator_agent)
_ = await runner.run_debug("Create a short video about teachers at university.")


 ### Created new session: debug_session_id

User > Create a short video about teachers at university.
IdeaAgent > Show a day in the life of a university professor, highlighting the unexpected moments of inspiration and exhaustion that come with shaping young minds.
ValidationAgent > INAPPROPRIATE


In [10]:
runner = InMemoryRunner(agent=orchiestrator_agent)
_ = await runner.run_debug("Create a short video about teachers at kindergarten.")


 ### Created new session: debug_session_id

User > Create a short video about teachers at kindergarten.
IdeaAgent > A sped-up montage of a kindergarten teacher's day, showcasing the chaos, creativity, and pure joy of shaping young minds.
ValidationAgent > APPROPRIATE
ActorsAgent > Alright, let's find some talent for this! A sped-up montage of a kindergarten teacher's day sounds like a blast – capturing that perfect blend of delightful chaos and heartwarming moments. We need actors who can bring energy, warmth, and that special spark of patience and playfulness.

Here are my top picks for this role:

**For the Lead Kindergarten Teacher:**

*   **Kristen Bell:** She has an incredible ability to convey both high energy and deep empathy. Her voice work (like Anna in *Frozen*) shows a fantastic range for engaging with children, and her comedic timing would be perfect for the "chaos" moments.
*   **Aubrey Plaza:** While known for more deadpan roles, she has a surprising warmth and a knack f